# Parsing a CSGO demofile¶
##### Last Updated: November 3, 2020

The csgo package allows for the parsing of CSGO demofiles into JSON or tabular format, which makes for easy analysis. To install the package, clone the Github repository, navigate to the root directory and run `python setup.py install`. Make sure you have installed at least [Golang](https://golang.org/) version 1.14.

##### What is a demofile?
Each CSGO match typically generates recordings of the game called a demofile. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined tick rate, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends. For a detailed overview of CSGO demofiles, see [1].

Since CSGO demofiles are essentially data transferred between the clients and the game server, the data is simply stored as a text of sequential set of events with no contextual information, such as round or map location. Thus, due to the highly unstructured nature of these low level CSGO data streams, akin to log files, performing any complex analytic tasks becomes impossible without modeling this data formally into a useful data structure. The csgo package addresses this need.

##### How do I get demofiles?¶
One of the most common ways to obtain demofiles is through the site [HLTV](https://hltv.org). If you navigate to a match page, an hour or so after the match has concluded, it will often contain a link to the demofiles for that match. The link is a `.rar` compressed directory containing a demo for each map in the match.

# Initializing the parser
Let's consider the demo from a match between OG and Natus Vincere (NaVi), which we can [download from HLTV](https://www.hltv.org/matches/2344822/og-vs-natus-vincere-blast-premier-fall-series-2020). If we download the compressed demofile directory (503 MB), there are three files:

- `og-vs-natus-vincere-m1-dust2.dem`
- `og-vs-natus-vincere-m2-mirage.dem`
- `og-vs-natus-vincere-m3-inferno.dem`

In order to parse one of the demofiles, you have to instantiate a DemoParser object and pass it the path to the demofile, along with a match_id. We will use the first demo, on `de_dust2`, and we will use a `demo_id` of `OG-NaVi-BLAST2020`. We will also specify a `parse_rate` of 128. The `parse_rate` is a power of 2 between $2^0$ and $2^7$, and indicates the spacing between parsed frames. A higher parse_rate indicates less output. For example, if `parse_rate = 128`, there is 1 second between recorded game frames. One can also write a log to disk by specifying `log = True`. The parsing computation can take a few seconds. Our output is a Python dictionary, which we inspect later. Additionally, the parser writes to disk a file named `demoId_mapName.json`.

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(demofile = "og-vs-natus-vincere-m1-dust2.dem", demo_id = "OG-NaVi-BLAST2020", parse_rate=128)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()

22:59:04 [INFO] Go version>=1.14.0
22:59:04 [INFO] Initialized CSGODemoParser with demofile og-vs-natus-vincere-m1-dust2.dem
22:59:04 [INFO] Setting demo id to OG-NaVi-BLAST2020
22:59:04 [INFO] Setting parse rate to 128
22:59:04 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
22:59:04 [INFO] Looking for file at /home/peter/Downloads/csgo_notebooks/og-vs-natus-vincere-m1-dust2.dem
22:59:24 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
22:59:24 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
22:59:24 [INFO] JSON data loaded, available in the `json` attribute to parser
22:59:24 [INFO] Successfully parsed JSON output
22:59:24 [INFO] Successfully returned JSON output


# Accessing the parsed data
Next, we can access our parsed data. The default data output is the JSON-like format of a Python dictionary. We can see the structure below:

```
├── MatchId (equal to demo_id)
├── ClientName
├── MapName
├── TickRate
├── PlayBackTicks
├── ParseRate
└── GameRounds (list of game rounds)
    ├── RoundNum
    ├── StartTick
    ├── FreezeTimeEnd
    ├── EndTick
    ├── ScoreUpdatedTick
    ├── EndOfficialTick
    ├── TScore
    ├── CTScore
    ├── WinningSide
    ├── WinningTeam
    ├── LosingTeam
    ├── RoundEndReason
    ├── CTStartEqVal
    ├── CTBuyType
    ├── TStartEqVal
    ├── TBuyType
    ├── Kills (list of events)
    ├── Damages (list of events)
    ├── Grenades (list of events)
    ├── BombEvents (list of events)
    ├── WeaponFires (list of events)
    ├── Flashes (list of events)
    └── Frames (list of events)
        ├── Tick
        ├── Second
        ├── PositionToken
        ├── TToken
        ├── CTToken
        ├── T
        │   ├── Side
        │   ├── TeamName
        │   ├── TeamEqVal
        │   ├── PositionToken
        │   ├── AlivePlayers
        │   ├── TotalUtility
        │   ├── UtilityLevel
        │   ├── Players (list of player information)
        │   │   ├── SteamId
        │   │   ├── Name
        │   │   ├── X
        │   │   ├── Y
        │   │   ├── Z
        │   │   ├── ViewX
        │   │   ├── ViewY
        │   │   ├── AreaId
        │   │   ├── AreaName
        │   │   ├── Hp
        │   │   ├── Armor
        │   │   ├── ActiveWeapon
        │   │   ├── TotalUtility
        │   │   ├── IsAlive
        │   │   ├── IsFlashed
        │   │   ├── IsAirborne
        │   │   ├── IsDucking
        │   │   ├── IsScoped
        │   │   ├── IsWalking
        │   │   ├── Inventory
        │   │   ├── EquipmentValue
        │   │   ├── Money
        │   │   ├── HasHelmet
        │   │   ├── HasDefuse
        │   │   ├── DistToBombsiteA
        │   └── └── DistToBombsiteB
        ├── CT (like T)
        ├── World (list of objects like bombs, grenades)
        ├── BombDistanceToA
        └── BombDistanceToB
```

In [2]:
for k in data.keys():
    if k != "GameRounds":
        print(k + ": " + str(data[k]))

MatchId: OG-NaVi-BLAST2020
ClientName: GOTV Demo
MapName: de_dust2
TickRate: 127
PlaybackTicks: 466670
ParseRate: 128


In [3]:
for k in data["GameRounds"][0].keys():
    if k not in ["Kills", "Damages", "Grenades", "BombEvents", "WeaponFires", "Flashes", "Frames"]:
        print(k + ": " + str(data["GameRounds"][0][k]))

RoundNum: 1
StartTick: 9308
FreezeTimeEnd: 37043
EndTick: 43177
ScoreUpdatedTick: 43208
EndOfficialTick: 43817
TScore: 0
CTScore: 0
WinningSide: CT
WinningTeam: OGEsports
LosingTeam: Natus Vincere
RoundEndReason: CTWin
CTStartEqVal: 4400
CTBuyType: Pistol
TStartEqVal: 4250
TBuyType: Pistol


## Event Data
The parser contains data on kills, damages, grenades, bomb events, weapon fires and flashes, which we show below, in order.

In [4]:
for k in data["GameRounds"][0]["Kills"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Kills"][0][k]))

Tick: 39106
Second: 16.244094488188978
AttackerSteamId: 76561198016432560
AttackerName: mantuu
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -294.1557312011719
AttackerY: 626.77783203125
AttackerZ: 1.0166358947753906
AttackerAreaId: 1713
AttackerAreaName: TopofMid
AttackerViewX: 319.5758056640625
AttackerViewY: 0.5987548828125
VictimSteamId: 76561198146207066
VictimName: Boombl4
VictimTeam: Natus Vincere
VictimSide: T
VictimX: 66.37433624267578
VictimY: 308.03125
VictimZ: 0.4415321350097656
VictimAreaId: 67
VictimAreaName: TopofMid
VictimViewX: 128.1170654296875
VictimViewY: 1.6534423828125
AssisterSteamId: 76561198013243326
AssisterName: Aleksib
AssisterTeam: OGEsports
AssisterSide: CT
AssisterX: -354.3523864746094
AssisterY: 1997.0689697265625
AssisterZ: -126.8095932006836
AssisterAreaId: 5222
AssisterAreaName: MidDoors
IsWallbang: False
IsFlashed: False
IsHeadshot: True
IsTrade: False
IsFirstKill: True
Weapon: USP-S


In [5]:
for k in data["GameRounds"][0]["Damages"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Damages"][0][k]))

Tick: 39015
Second: 15.527559055118111
AttackerSteamId: 76561198016432560
AttackerName: mantuu
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -284.76715087890625
AttackerY: 632.74462890625
AttackerZ: 1.266963243484497
AttackerAreaId: 1713
AttackerAreaName: TopofMid
AttackerViewX: 314.307861328125
AttackerViewY: 0.50537109375
VictimSteamId: 76561198146207066
VictimName: Boombl4
VictimTeam: Natus Vincere
VictimSide: T
VictimX: 48.46458435058594
VictimY: 308.03125
VictimZ: 0.4415321350097656
VictimAreaId: 67
VictimAreaName: TopofMid
VictimViewX: 39.495849609375
VictimViewY: 1.944580078125
Weapon: USP-S
HpDamage: 32
KillHpDamage: 32
ArmorDamage: 0
HitGroup: Unknown


In [6]:
for k in data["GameRounds"][0]["Grenades"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Grenades"][0][k]))

Tick: 38142
Second: 8.653543307086615
PlayerSteamId: 76561198121220486
PlayerName: Perfecto
PlayerTeam: Natus Vincere
PlayerSide: T
PlayerX: -1050.7694091796875
PlayerY: -675.33251953125
PlayerZ: 128.14610290527344
PlayerAreaId: 8828
PlayerAreaName: TSpawn
GrenadeType: Smoke Grenade
GrenadeX: -275.00665283203125
GrenadeY: 1413.9776611328125
GrenadeZ: -26.495166778564453
GrenadeAreaId: 7822
GrenadeAreaName: Middle


In [7]:
for k in data["GameRounds"][6]["BombEvents"][0].keys():
        print(k + ": " + str(data["GameRounds"][6]["BombEvents"][0][k]))

Tick: 129543
Second: 68.05511811023622
PlayerSteamId: 76561198034202275
PlayerName: s1mple
PlayerTeam: Natus Vincere
BombAction: plant
BombSite: A


In [8]:
for k in data["GameRounds"][0]["WeaponFires"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["WeaponFires"][0][k]))

Tick: 37440
Second: 3.125984251968504
PlayerSteamId: 76561198121220486
PlayerName: Perfecto
PlayerTeam: Natus Vincere
PlayerSide: T
PlayerX: -503.520263671875
PlayerY: -661.4535522460938
PlayerZ: 127.1343002319336
PlayerAreaId: 2000
PlayerAreaName: TSpawn
PlayerViewX: 84.2376708984375
PlayerViewY: 354.0948486328125
WeaponName: Smoke Grenade


In [9]:
for k in data["GameRounds"][0]["Flashes"][0].keys():
        print(k + ": " + str(data["GameRounds"][0]["Flashes"][0][k]))

Tick: 38933
Second: 14.881889763779528
AttackerSteamId: 76561198013243326
AttackerName: Aleksib
AttackerTeam: OGEsports
AttackerSide: CT
AttackerX: -490.8686218261719
AttackerY: 1940.558349609375
AttackerZ: -122.56611633300781
AttackerAreaId: 5228
AttackerAreaName: MidDoors
AttackerViewX: 187.503662109375
AttackerViewY: 5.91064453125
PlayerSteamId: 76561198013243326
PlayerName: Aleksib
PlayerTeam: OGEsports
PlayerSide: CT
PlayerX: -490.8686218261719
PlayerY: 1940.558349609375
PlayerZ: -122.56611633300781
PlayerAreaId: 5228
PlayerAreaName: MidDoors
PlayerViewX: 187.503662109375
PlayerViewY: 5.91064453125


## Event Data as DataFrames

In [10]:
demo_parser._parse_rounds(return_type="df")

22:59:25 [INFO] Parsed rounds to Pandas DataFrame


,RoundNum,MatchId,MapName,StartTick,FreezeTimeEnd,EndTick,EndOfficialTick,TScore,CTScore,WinningSide,WinningTeam,LosingTeam,RoundEndReason,CTStartEqVal,CTBuyType,TStartEqVal,TBuyType
0,1,OG-NaVi-BLAST2020,de_dust2,9308,37043,43177,43817,0,0,CT,OGEsports,Natus Vincere,CTWin,4400,Pistol,4250,Pistol
1,2,OG-NaVi-BLAST2020,de_dust2,43817,46377,59967,60608,0,1,T,Natus Vincere,OGEsports,TerroristsWin,21200,Half Buy,10400,Half Buy
2,3,OG-NaVi-BLAST2020,de_dust2,60608,63168,72870,73511,1,1,T,Natus Vincere,OGEsports,TerroristsWin,8050,Half Buy,23600,Full Buy
3,4,OG-NaVi-BLAST2020,de_dust2,73511,76071,84021,84662,2,1,T,Natus Vincere,OGEsports,TerroristsWin,1000,Full Eco,22450,Full Buy
4,5,OG-NaVi-BLAST2020,de_dust2,84662,87222,99779,100420,3,1,CT,OGEsports,Natus Vincere,CTWin,22600,Full Buy,26850,Full Buy
5,6,OG-NaVi-BLAST2020,de_dust2,100420,102980,117700,118340,3,2,CT,OGEsports,Natus Vincere,TargetSaved,21950,Half Buy,26750,Full Buy
6,7,OG-NaVi-BLAST2020,de_dust2,118340,120900,134791,135431,3,3,T,Natus Vincere,OGEsports,TargetBombed,29050,Full Buy,24850,Full Buy
7,8,OG-NaVi-BLAST2020,de_dust2,135431,137991,152711,153351,4,3,CT,OGEsports,Natus Vincere,TargetSaved,21750,Half Buy,25750,Full Buy
8,9,OG-NaVi-BLAST2020,de_dust2,153351,155911,168760,169400,4,4,CT,OGEsports,Natus Vincere,CTWin,27450,Full Buy,19950,Full Buy
9,10,OG-NaVi-BLAST2020,de_dust2,169400,171960,181220,181861,4,5,CT,OGEsports,Natus Vincere,CTWin,29100,Full Buy,1500,Full Eco


In [11]:
demo_parser._parse_kills(return_type="df")

22:59:25 [INFO] Parsed kills to Pandas DataFrame


,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,AssisterAreaName,IsWallbang,IsFlashed,IsHeadshot,IsTrade,IsFirstKill,Weapon,RoundNum,MatchId,MapName
0,39106,16.244094,7.656120e+16,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,1713.0,...,MidDoors,False,False,True,False,True,USP-S,1,OG-NaVi-BLAST2020,de_dust2
1,39549,19.732283,7.656120e+16,ISSAA,OGEsports,CT,-1845.398804,2719.539307,32.390869,6887.0,...,None,False,False,True,False,False,USP-S,1,OG-NaVi-BLAST2020,de_dust2
2,39632,20.385827,7.656120e+16,mantuu,OGEsports,CT,-171.508743,761.183167,2.132068,1082.0,...,None,False,False,True,False,False,USP-S,1,OG-NaVi-BLAST2020,de_dust2
3,39696,20.889764,7.656120e+16,flamie,Natus Vincere,T,-604.332642,1334.031250,-108.254135,1287.0,...,None,False,False,True,False,False,Glock-18,1,OG-NaVi-BLAST2020,de_dust2
4,40045,23.637795,7.656120e+16,mantuu,OGEsports,CT,-149.031250,1001.231262,1.489925,1083.0,...,None,False,False,True,True,False,USP-S,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,456139,56.866142,7.656120e+16,electronic,Natus Vincere,CT,-484.418335,1847.642334,-123.553299,5220.0,...,None,False,False,True,False,False,M4A4,25,OG-NaVi-BLAST2020,de_dust2
159,458062,72.007874,7.656120e+16,valde,OGEsports,T,-2025.999146,1511.807251,33.910797,8063.0,...,None,False,False,True,False,False,AK-47,25,OG-NaVi-BLAST2020,de_dust2
160,458223,73.275591,7.656120e+16,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,8063.0,...,None,False,False,True,False,False,AK-47,25,OG-NaVi-BLAST2020,de_dust2
161,458294,73.834646,7.656120e+16,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,1245.0,...,UpperTunnel,False,False,True,True,False,M4A4,25,OG-NaVi-BLAST2020,de_dust2


In [12]:
demo_parser._parse_damages(return_type="df")

22:59:25 [INFO] Parsed damages to Pandas DataFrame


,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,VictimViewX,VictimViewY,Weapon,HpDamage,KillHpDamage,ArmorDamage,HitGroup,RoundNum,MatchId,MapName
0,39015,15.527559,7.656120e+16,mantuu,OGEsports,CT,-284.767151,632.744629,1.266963,1713.0,...,39.495850,1.944580,USP-S,32,32,0,Unknown,1,OG-NaVi-BLAST2020,de_dust2
1,39072,15.976378,7.656120e+16,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,1713.0,...,89.961548,3.059692,USP-S,31,31,0,RightArm,1,OG-NaVi-BLAST2020,de_dust2
2,39106,16.244094,7.656120e+16,mantuu,OGEsports,CT,-294.155731,626.777832,1.016636,1713.0,...,128.117065,1.653442,USP-S,127,37,0,Head,1,OG-NaVi-BLAST2020,de_dust2
3,39526,19.551181,7.656120e+16,electronic,Natus Vincere,T,-579.786072,1494.131836,-110.432953,1287.0,...,119.146729,8.541870,Glock-18,91,91,0,Head,1,OG-NaVi-BLAST2020,de_dust2
4,39549,19.732283,7.656120e+16,ISSAA,OGEsports,CT,-1845.398804,2719.539307,32.390869,6887.0,...,86.797485,0.472412,USP-S,106,100,0,Head,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,458191,73.023622,7.656120e+16,Boombl4,Natus Vincere,CT,-1873.927979,1302.483154,33.934990,1230.0,...,310.259399,0.351562,AK-47,27,27,4,Chest,25,OG-NaVi-BLAST2020,de_dust2
700,458217,73.228346,7.656120e+16,Boombl4,Natus Vincere,CT,-1883.084595,1294.227783,32.957306,1230.0,...,309.776001,0.527344,AK-47,27,27,4,Chest,25,OG-NaVi-BLAST2020,de_dust2
701,458223,73.275591,7.656120e+16,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,8063.0,...,119.712524,2.614746,AK-47,110,100,16,Head,25,OG-NaVi-BLAST2020,de_dust2
702,458294,73.834646,7.656120e+16,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,1245.0,...,295.208130,0.834961,M4A4,89,46,19,Head,25,OG-NaVi-BLAST2020,de_dust2


In [13]:
demo_parser._parse_grenades(return_type="df")

22:59:25 [INFO] Parsed grenades to Pandas DataFrame


,Tick,Second,PlayerSteamId,PlayerName,PlayerTeam,PlayerSide,PlayerX,PlayerY,PlayerZ,PlayerAreaId,PlayerAreaName,GrenadeType,GrenadeX,GrenadeY,GrenadeZ,GrenadeAreaId,GrenadeAreaName,RoundNum,MatchId,MapName
0,38142,8.653543,76561198121220486,Perfecto,Natus Vincere,T,-1050.769409,-675.332520,128.146103,8828,TSpawn,Smoke Grenade,-275.006653,1413.977661,-26.495167,7822,Middle,1,OG-NaVi-BLAST2020,de_dust2
1,38933,14.881890,76561198013243326,Aleksib,OGEsports,CT,-491.269257,1939.669434,-122.541168,5228,MidDoors,Flashbang,-148.543335,509.083679,91.145599,8109,TopofMid,1,OG-NaVi-BLAST2020,de_dust2
2,40453,26.850394,76561198121220486,Perfecto,Natus Vincere,T,-2055.255859,1206.173706,33.546051,8220,UpperTunnel,Smoke Grenade,-275.006653,1413.977661,-26.495167,7822,Middle,1,OG-NaVi-BLAST2020,de_dust2
3,46654,2.181102,76561198016432560,mantuu,OGEsports,CT,650.234314,2238.515869,-82.071518,8670,UnderA,Smoke Grenade,-356.675873,2234.270752,-121.740883,8492,MidDoors,2,OG-NaVi-BLAST2020,de_dust2
4,47439,8.362205,76561198116523276,flamie,Natus Vincere,T,413.240082,-174.923615,-4.058386,8846,OutsideLong,Flashbang,1190.807861,645.778809,552.034363,7684,LongDoors,2,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,456048,56.149606,76561198114929868,valde,OGEsports,T,-1669.058960,1762.856934,1.867692,1652,BombsiteB,Flashbang,-1122.075806,2277.406250,244.030746,1899,BDoors,25,OG-NaVi-BLAST2020,de_dust2
701,456326,58.338583,76561198114929868,valde,OGEsports,T,-1700.344971,1788.224243,4.340763,1650,BombsiteB,Flashbang,-1162.880615,2282.995605,267.011810,6673,BDoors,25,OG-NaVi-BLAST2020,de_dust2
702,457208,65.283465,76561198114929868,valde,OGEsports,T,-2025.998047,1494.800415,33.902771,8062,UpperTunnel,Smoke Grenade,-1265.867065,2224.500488,4.576219,6679,BDoors,25,OG-NaVi-BLAST2020,de_dust2
703,457761,69.637795,76561197960710573,NBK-,OGEsports,T,-1944.478760,2432.775146,18.963537,6815,BombsiteB,Molotov,-1280.750000,2645.343750,122.906250,8658,Hole,25,OG-NaVi-BLAST2020,de_dust2


In [14]:
demo_parser._parse_bomb_events(return_type="df")

22:59:25 [INFO] Parsed bomb_events to Pandas DataFrame


,Tick,Second,PlayerSteamId,PlayerName,PlayerTeam,BombAction,BombSite,RoundNum,MatchId,MapName
0,56729,81.511811,76561198044045107,electronic,Natus Vincere,plant,B,2,OG-NaVi-BLAST2020,de_dust2
1,72831,76.086614,76561198116523276,flamie,Natus Vincere,plant,A,3,OG-NaVi-BLAST2020,de_dust2
2,81661,44.015748,76561198116523276,flamie,Natus Vincere,plant,B,4,OG-NaVi-BLAST2020,de_dust2
3,129543,68.055118,76561198034202275,s1mple,Natus Vincere,plant,A,7,OG-NaVi-BLAST2020,de_dust2
4,234011,30.370079,76561198034202275,s1mple,Natus Vincere,plant,B,14,OG-NaVi-BLAST2020,de_dust2
5,254114,91.771654,76561198121220486,Perfecto,Natus Vincere,plant,A,15,OG-NaVi-BLAST2020,de_dust2
6,289400,36.763780,76561198037812456,ISSAA,OGEsports,plant,A,16,OG-NaVi-BLAST2020,de_dust2
7,294222,74.732283,76561198044045107,electronic,Natus Vincere,defuse,A,16,OG-NaVi-BLAST2020,de_dust2
8,315992,103.535433,76561198016432560,mantuu,OGEsports,plant,B,17,OG-NaVi-BLAST2020,de_dust2
9,335007,108.354331,76561198013243326,Aleksib,OGEsports,plant,A,18,OG-NaVi-BLAST2020,de_dust2


In [15]:
demo_parser._parse_flashes(return_type="df")

22:59:25 [INFO] Parsed flashes to Pandas DataFrame


,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,PlayerX,PlayerY,PlayerZ,PlayerAreaId,PlayerAreaName,PlayerViewX,PlayerViewY,RoundNum,MatchId,MapName
0,38933,14.881890,76561198013243326,Aleksib,OGEsports,CT,-490.868622,1940.558350,-122.566116,5228,...,-490.868622,1940.558350,-122.566116,5228,MidDoors,187.503662,5.910645,1,OG-NaVi-BLAST2020,de_dust2
1,38933,14.881890,76561198013243326,Aleksib,OGEsports,CT,-490.868622,1940.558350,-122.566116,5228,...,-169.062500,721.509399,1.603693,1082,Catwalk,25.312500,0.087891,1,OG-NaVi-BLAST2020,de_dust2
2,38933,14.881890,76561198013243326,Aleksib,OGEsports,CT,-490.868622,1940.558350,-122.566116,5228,...,-58.232361,315.078949,1.081378,67,TopofMid,104.128418,0.955811,1,OG-NaVi-BLAST2020,de_dust2
3,47439,8.362205,76561198116523276,flamie,Natus Vincere,T,414.144012,-173.192261,-4.092491,8846,...,1335.441528,1068.796509,-7.313267,4191,LongA,199.215088,2.026978,2,OG-NaVi-BLAST2020,de_dust2
4,47439,8.362205,76561198116523276,flamie,Natus Vincere,T,414.144012,-173.192261,-4.092491,8846,...,1336.562988,1441.839966,-5.035785,4184,LongA,225.615234,4.015503,2,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,455965,55.496063,76561198146207066,Boombl4,Natus Vincere,CT,-460.862274,597.853394,-0.781033,1713,...,-1638.507812,2598.031250,3.781982,1572,BombsiteB,293.510742,5.960083,25,OG-NaVi-BLAST2020,de_dust2
613,456048,56.149606,76561198114929868,valde,OGEsports,T,-1668.181763,1762.089966,1.801992,1652,...,-1654.943481,1645.333618,2.342754,1642,BombsiteB,208.064575,0.131836,25,OG-NaVi-BLAST2020,de_dust2
614,456048,56.149606,76561198114929868,valde,OGEsports,T,-1668.181763,1762.089966,1.801992,1652,...,-27.620670,2208.076660,-129.532272,1802,CTSpawn,198.616333,359.593506,25,OG-NaVi-BLAST2020,de_dust2
615,456326,58.338583,76561198114929868,valde,OGEsports,T,-1700.748169,1789.433228,4.254478,1650,...,-1700.748169,1789.433228,4.254478,1650,BombsiteB,114.417114,6.377563,25,OG-NaVi-BLAST2020,de_dust2
